Part 1: Climate Analysis and Exploration
In this section, you’ll use Python and SQLAlchemy to perform basic climate analysis and data exploration of your climate database. Complete the following tasks by using SQLAlchemy ORM queries, Pandas, and Matplotlib.


Use the provided starter notebook and hawaii.sqlite files to complete your climate analysis and data exploration.


Use SQLAlchemy’s create_engine to connect to your SQLite database.


Use SQLAlchemy’s automap_base() to reflect your tables into classes and save a reference to those classes called Station and Measurement.


Link Python to the database by creating a SQLAlchemy session.


Important: Don't forget to close out your session at the end of your notebook.

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import create_engine, inspect, func
from sqlalchemy import distinct

import matplotlib
from matplotlib import style
import matplotlib.pyplot as plt
import pandas as pd

from datetime import datetime
from dateutil. relativedelta import relativedelta
import numpy as np



In [ ]:
# Create engine using the database file
engine = create_engine(f"sqlite:////Users/swa/Documents/UC_Irvine/Homework/10-Advanced-Data-Storage-and-Retrieval/Instructions/Resources/hawaii.sqlite") 
conn = engine.connect()

# Declare a Base using `automap_base()`
Base = automap_base()

# Use the Base class to reflect the database tables
Base.prepare(engine, reflect=True)

# Print all of the classes mapped to the Base
Base.classes.keys()

# Assign the classes to variables called Station and Measurement
Station = Base.classes.station
Measurement = Base.classes.measurement

# Create a session
session = Session(engine)

Precipitation Analysis
To perform an analysis of precipitation in the area, do the following:


Find the most recent date in the dataset.


Using this date, retrieve the previous 12 months of precipitation data by querying the 12 previous months of data. Note: Do not pass in the date as a variable to your query.


Select only the date and prcp values.


Load the query results into a Pandas DataFrame, and set the index to the date column.


Sort the DataFrame values by date.


Plot the results by using the DataFrame plot method, as shown in the following image:


Use Pandas to print the summary statistics for the precipitation data.


In [ ]:
# Find the most recent date in the dataset.
# Using this date, retrieve the previous 12 months of precipitation data by querying the 12 previous months of data. Note: Do not pass in the date as a variable to your query.
measurements = session.query(Measurement)
most_recent_date = measurements[-1].date
previous_year = datetime.fromisoformat(most_recent_date) - relativedelta(years=1)

measurements = session.query(Measurement).\
    filter(Measurement.date >= previous_year).\
    group_by(Measurement.date).\
    order_by(Measurement.date).all()

# Select only the date and prcp values.
df = pd.DataFrame(columns = ["Date", "PRCP"])
for measurement in measurements:
    df = df.append({'Date' : measurement.date, 'PRCP' : measurement.prcp}, ignore_index=True)

x = np.array(df["Date"].values)
y = np.array(df["PRCP"].values)
plt.bar(x, y)
plt.title("Precipitation")
plt.xlabel("Date")
plt.ylabel("Precipitation")
plt.show()

df.describe()


Station Analysis
To perform an analysis of stations in the area, do the following:


Design a query to calculate the total number of stations in the dataset.


Design a query to find the most active stations (the stations with the most rows).


List the stations and observation counts in descending order.


Which station id has the highest number of observations?


Using the most active station id, calculate the lowest, highest, and average temperatures.


Hint: You will need to use functions such as func.min, func.max, func.avg, and func.count in your queries.

In [ ]:
# Design a query to calculate the total number of stations in the dataset.
stations = session.query(Station.name).count()
print("Total number of stations " + str(stations))

# Design a query to find the most active stations (the stations with the most rows).
measurements = session.query(Measurement.station).\
        group_by(Measurement.station)

# List the stations and observation counts in descending order.
stations = session.query(Measurement.station, func.count(Measurement.station)).\
        group_by(Measurement.station).\
        order_by(func.count(Measurement.station).desc()).all()

# Which station id has the highest number of observations?
top_station = stations[0][0]
print("Top Station is " + str(top_station))

# Using the most active station id, calculate the lowest, highest, and average temperatures.
max_temp = session.query(Measurement.station, func.max(Measurement.prcp)).\
    filter(Measurement.station == top_station).all()
max_temp_value = max_temp[0][1]
print("Max Temperature is " + str(max_temp_value))

min_temp = session.query(Measurement.station, func.min(Measurement.prcp)).\
    filter(Measurement.station == top_station).all()
min_temp_value = min_temp[0][1]
print("Min Temperature is " + str(min_temp_value))

average_temp = session.query(Measurement.station, func.avg(Measurement.prcp)).\
    filter(Measurement.station == top_station).all()
avg_temp_value = average_temp[0][1]
print("Average Temperature is " + str(avg_temp_value))


Design a query to retrieve the previous 12 months of temperature observation data (TOBS).


Filter by the station with the highest number of observations.


Query the previous 12 months of temperature observation data for this station.


Plot the results as a histogram with bins=12, as shown in the following image


Close out your session.

In [ ]:
# Design a query to retrieve the previous 12 months of temperature observation data (TOBS).
measurements = session.query(Measurement).\
    filter(Measurement.date >= previous_year).\
    group_by(Measurement.date).\
    order_by(Measurement.date).all()

# Filter by the station with the highest number of observations.
stations = session.query(Measurement.station, func.count(Measurement.station)).\
    filter(Measurement.date >= previous_year).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.station).desc()).all()

top_station = stations[0][0]

# Query the previous 12 months of temperature observation data for this station.
top_measurements = session.query(Measurement).\
    filter(Measurement.date >= previous_year).\
    filter(Measurement.station == top_station).\
    group_by(Measurement.date).\
    order_by(Measurement.date).all()

# Which station id has the highest number of observations?
print("Top Station is " + str(top_station))

# Plot the results as a histogram with bins=12, as shown in the following image
temperatures = []
for measurement in top_measurements:
    temperatures.append(float(measurement.prcp or 0))
plt.hist(temperatures)
plt.show()